# Sereen — One‑Click Colab (Face‑Locked WebUI w/ ReActor)

This version is hardened to avoid the **tokenizers** build error. It:
- Prefers prebuilt wheels (no source build)
- Falls back to installing Rust **only if needed**
- Launches a **public URL** via Cloudflare (more reliable than `--share`)

**How to run**
1. `Runtime → Change runtime type → GPU (T4 is fine) → Save`
2. `Runtime → Run all` (first run ~8–15 minutes)
3. Click the printed **public URL** at the end (…`trycloudflare.com`)
4. In WebUI → **Extensions → ReActor**: upload **Sereen_01** → Swap **0.7** → Restore face **ON** → Save
5. Go to **txt2img** and generate using your prompts.


In [ ]:
# 0) Harden deps to avoid tokenizers build from source (with fallback)
# Try prebuilt wheels; if unavailable, auto-install Rust toolchain then retry.
!pip -q install --upgrade pip setuptools wheel
failed=0
try:
    import subprocess, sys
    cmd = [sys.executable, "-m", "pip", "install", "-q", "--only-binary", ":all:",
           "tokenizers==0.15.2", "transformers==4.35.2", "accelerate==0.24.1", "safetensors>=0.4.2", "einops"]
    subprocess.check_call(cmd)
except Exception as e:
    failed=1

if failed:
    import os, subprocess, sys
    print("Prebuilt wheels not available — installing Rust toolchain and retrying…")
    os.system("apt -y update -qq && apt -y install -qq rustc cargo build-essential")
    cmd = [sys.executable, "-m", "pip", "install", "-q",
           "tokenizers==0.15.2", "transformers==4.35.2", "accelerate==0.24.1", "safetensors>=0.4.2", "einops"]
    subprocess.check_call(cmd)
print("✅ Dependencies prepared.")


In [ ]:
# 1) Install WebUI + ReActor and download SD 1.5 model
!apt -y update -qq
!apt -y install -qq git aria2
%cd /content

# Clone AUTOMATIC1111 WebUI
!git clone -q https://github.com/AUTOMATIC1111/stable-diffusion-webui.git

# Add ReActor (face-locking) extension
%cd /content/stable-diffusion-webui/extensions
!git clone -q https://github.com/Gourieff/sd-webui-reactor.git

# Download SD 1.5 model (public, no token needed)
%cd /content/stable-diffusion-webui/models/Stable-diffusion
!aria2c -q -x16 -s16 -k1M -o v1-5-pruned-emaonly.ckpt \
  https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt
print("✅ Model ready.")


In [ ]:
# 2) Launch with reliable public URL (Cloudflare tunnel)
%cd /content/stable-diffusion-webui
!pip -q install cloudflared
print("Launching WebUI… watch for: Running on public URL: https://***.trycloudflare.com")
# Force pip to prefer binary wheels during WebUI's own dependency installs
!PIP_ONLY_BINARY=:all: COMMANDLINE_ARGS="--xformers --enable-insecure-extension-access --gradio-queue --theme=dark --cloudflared" \
REQUIREMENTS_FILE=requirements_versions.txt \
python launch.py


## Prompts
Use your `Sereen_Prompts.txt` or paste one of these to test:
- *Sereen near black‑framed windows, cream satin long‑sleeve midi dress, modest neckline, minimal decor, natural light, soft feminine glow*
- *Sereen in an aesthetic closet with only dresses on a rack, belted long‑sleeve dress, neutral palette, soft light*
- *Sereen at a minimalist vanity, modest satin robe, applying serum, clean beauty mood, daylight*
